## D213 Advanced Data Analytics PA 1
##### Submitted By Edwin Perry
### Table of Contents
<ol>
    <li><a href="#A">Research Question</a></li>
    <li><a href="#B">Data Preparation</a></li>
    <li><a href="#C">Network Architecture</a></li>
    <li><a href="#D">Model Evaluation</a></li>
    <li><a href="#E">Summary and Recommendations</a></li>
    <li><a href="#F">Reporting</a></li>
</ol>
<h4 id="A">Research Question</h4>